In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# df = pd.read_csv("/kaggle/input/ai4digigov2021/queue_dataset_train.csv")
# df["service_name"].value_counts()
# df["service_name_2"].value_counts()

In [11]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import gc
import seaborn as sns
import xgboost as xgb
from xgboost import XGBClassifier
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, classification_report, roc_auc_score
import matplotlib.dates as mdates
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.graph_objs import Line
import plotly.express as px
import catboost
import seaborn as sn
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 300)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# plot sizes
plt.rcParams["figure.figsize"] = [15, 10]

In [12]:
# --- Read Data ---
q_df = pd.read_csv("/kaggle/input/ai4digigov2021/queue_dataset_train.csv")
q_df_test = pd.read_csv("/kaggle/input/ai4digigov2021/queue_dataset_test.csv")
q_df_sub = pd.read_csv("/kaggle/input/ai4digigov2021/baseline_submission.csv")

# EDA

In [ ]:
q_df.info()

In [13]:
# --- Convert date into pandas date ---
q_df["date_converted"] = pd.to_datetime(q_df["date"], format='%Y-%m-%d')
q_df_test["date_converted"] = pd.to_datetime(q_df_test["date"], format='%Y-%m-%d')

In [14]:
# --- Holiday Extractor
sorted_dates_train = sorted(q_df_test["date_converted"].dt.date.unique().tolist())
date_set = set(sorted_dates_train[0] + timedelta(x) for x in range((sorted_dates_train[-1] - sorted_dates_train[0]).days))

train_dataset_days_with_no_service = sorted(date_set - set(sorted_dates_train))
train_dataset_days_with_no_service

[datetime.date(2020, 1, 20), datetime.date(2020, 3, 8)]

In [ ]:
# --- Print NA ---
q_df.isna().sum()

In [15]:
# --- Impute NA values ---

# impute ages
q_df["customer_age_appl"].fillna(q_df["customer_age_appl"].mode().iloc[0], inplace=True)

# impute ticket hours
q_df["ticket_taking_time_hour"] = q_df['time_start_process'].str[:2] 
q_df["ticket_taking_time_hour"] = q_df["ticket_taking_time_hour"].astype(str)
q_df['ticket_taking_time_hour'].fillna(q_df['ticket_taking_time_hour'].mode().iloc[0], inplace= True)
q_df["ticket_taking_time_hour"] = q_df["ticket_taking_time_hour"].replace("nan", q_df['ticket_taking_time_hour'].mode().iloc[0])

In [16]:
# --- Extract Separate dates from date ---

# extract month 
q_df["approach_month"] = q_df["date_converted"].dt.month.astype(str)

# extract weekdays
week_days={0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
q_df['weekday'] = q_df['date_converted'].dt.dayofweek.map(week_days)

In [17]:
# --- Drop Columns ---

# - CORRELATION of [operator_count , previous_customer_count] 
#df = q_df[["operator_count","previous_customer_count"]]
#corrMatrix = df.corr()
#sn.heatmap(corrMatrix, annot=True)
#plt.show()
#RESULT: 0.43 corr --> DROP NONE

# - Already used, so UNNECESSARY columns dropped
# Note: dont drop service_name, instead drop service_name_2, because there are more varience in service_name
q_df.drop(["id", "date", "time_start_process", "date_converted"], axis=1, inplace=True) 

In [20]:
# ------------ Theirs Feature Selection for X -----------
# [["operator_count", "customer_gender", 'customer_age_appl',
# 'customer_city', "approach_month", "ticket_taking_time_hour"]]

# ------------ Our Feature Selection for X ------------
# dataset = q_df[["branch_name", "customer_gender", "customer_age_appl", "customer_city",
#                 "service_name_organization", "service_name", "service_name_2",
#                 "operator_count", "previous_customer_count",
#                "approach_month","ticket_taking_time_hour", "weekday"]]

# Model Selection & Hyperparameter Tuning

In [21]:
import sys
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import roc_auc_score
sys.path.insert(1, '../input/paramsearch/')

from paramsearch import paramsearch
from itertools import product,chain
from sklearn import metrics

## Grid Search + Cross Validation for Hyperparameter Tuning

In [22]:
# params = {'depth':[2,3,5],                 #'depth':[2,3,4,5],
#           'iterations':[250,500,1000],              #'iterations':[500,1000],
#           'learning_rate':[0.01,0.1,0.3],   #'learning_rate':[0.05, 0.1, 0.3], 
#           'l2_leaf_reg':[3,5],              #'l2_leaf_reg':[1,3,5],
#           'border_count':128,
#           'thread_count':4,
#           'task_type':"GPU",
#           'loss_function':'Logloss',
#           'eval_metric':"AUC",
#           'auto_class_weights':"Balanced",
#           'simple_ctr':None,
#           'verbose':True}

# def crossvaltest(params,splits=None):
#     skf = StratifiedKFold(n_splits=splits, shuffle=True, random_state=42)
#     skf_train_dev = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)
#     service = q_df['service_canceled']
#     res = []
#     for train_index, test_index in skf.split(q_df, service):
#         train = q_df.loc[train_index, :]
#         test = q_df.loc[test_index, :]
        
#         y_train = train['service_canceled'] 
#         x_train = train.drop(["service_canceled"], axis=1)
        
#         y_test = test['service_canceled']
#         x_test = test.drop(["service_canceled"], axis=1)

#         categorical_features_indices = [0,1,2,3,4,5,6,9,10,11]
#         train_pool = catboost.Pool(x_train, y_train, cat_features=categorical_features_indices)

#         model = catboost.CatBoostClassifier(**params)
#         model.fit(train_pool)
        
#         y_pred = model.predict_proba(x_test)[:, 1]
#         auc = roc_auc_score(y_test, y_pred)
#         res.append(auc)
#     return np.mean(res)

# def catboost_param_tune(params):
#     ps = paramsearch(params)
#     for prms in chain(ps.grid_search(['l2_leaf_reg']),
#                       ps.grid_search(['iterations','learning_rate']),
#                       ps.grid_search(['depth'])):
#         res = crossvaltest(prms,splits=4)
#         ps.register_result(res,prms)
#         print(f"{res},{prms},best:,{ps.bestscore()},{ps.bestparam()}")
#     return ps.bestparam()

# bestparams = catboost_param_tune(params)

In [ ]:
# bestparams
# clf = catboost.CatBoostClassifier(**bestparams)
# categorical_features_indices = [0,1,2,3,4,5,6,9,10,11]

# labels = q_df['service_canceled']
# dataset = q_df.drop(["service_canceled"], axis=1)
    
# train_pool = catboost.Pool(dataset, labels, cat_features=categorical_features_indices)
# clf.fit(train_pool)

## Ensemble Separate Catboot Models

In [23]:
bestparams = {'depth': 4,
 'iterations': 400,
 'learning_rate': 0.1,
 'l2_leaf_reg': 7,
 'border_count': 128,
 'thread_count': 4,
 'task_type': 'GPU',
 'loss_function': 'Logloss',
 'eval_metric': 'AUC',
 'auto_class_weights': 'Balanced',
 'simple_ctr': None,
 'verbose': True}


clf_1 = catboost.CatBoostClassifier(**bestparams)
categorical_features_indices = [0,1,2,3,4,5,6,9,10,11]

labels = q_df['service_canceled']
dataset = q_df.drop(["service_canceled"], axis=1)
    
train_pool = catboost.Pool(dataset, labels, cat_features=categorical_features_indices)
clf_1.fit(train_pool)

0:	learn: 0.6700959	total: 371ms	remaining: 2m 28s
1:	learn: 0.6752224	total: 714ms	remaining: 2m 22s
2:	learn: 0.6752462	total: 1.05s	remaining: 2m 18s
3:	learn: 0.6762584	total: 1.38s	remaining: 2m 16s
4:	learn: 0.6772932	total: 1.71s	remaining: 2m 15s
5:	learn: 0.6800142	total: 2.04s	remaining: 2m 14s
6:	learn: 0.6803879	total: 2.37s	remaining: 2m 12s
7:	learn: 0.6865997	total: 2.68s	remaining: 2m 11s
8:	learn: 0.6901869	total: 3s	remaining: 2m 10s
9:	learn: 0.6921726	total: 3.31s	remaining: 2m 9s
10:	learn: 0.6947327	total: 3.63s	remaining: 2m 8s
11:	learn: 0.6965767	total: 3.94s	remaining: 2m 7s
12:	learn: 0.6978894	total: 4.24s	remaining: 2m 6s
13:	learn: 0.6989799	total: 4.56s	remaining: 2m 5s
14:	learn: 0.7003807	total: 4.82s	remaining: 2m 3s
15:	learn: 0.7011842	total: 5.13s	remaining: 2m 3s
16:	learn: 0.7016372	total: 5.43s	remaining: 2m 2s
17:	learn: 0.7027591	total: 5.74s	remaining: 2m 1s
18:	learn: 0.7035157	total: 6.05s	remaining: 2m 1s
19:	learn: 0.7037891	total: 6.36s	r

In [24]:
bestparams = {'depth': 3,
 'iterations': 800,
 'learning_rate': 0.1,
 'l2_leaf_reg': 6,
 'border_count': 128,
 'thread_count': 4,
 'task_type': 'GPU',
 'loss_function': 'Logloss',
 'eval_metric': 'AUC',
 'auto_class_weights': 'Balanced',
 'simple_ctr': None,
 'verbose': True}


clf_2 = catboost.CatBoostClassifier(**bestparams)
categorical_features_indices = [0,1,2,3,4,5,6,9,10,11]

labels = q_df['service_canceled']
dataset = q_df.drop(["service_canceled"], axis=1)
    
train_pool = catboost.Pool(dataset, labels, cat_features=categorical_features_indices)
clf_2.fit(train_pool)

0:	learn: 0.6619775	total: 233ms	remaining: 3m 6s
1:	learn: 0.6650600	total: 470ms	remaining: 3m 7s
2:	learn: 0.6656708	total: 701ms	remaining: 3m 6s
3:	learn: 0.6656725	total: 937ms	remaining: 3m 6s
4:	learn: 0.6690260	total: 1.2s	remaining: 3m 11s
5:	learn: 0.6748237	total: 1.52s	remaining: 3m 21s
6:	learn: 0.6772493	total: 1.76s	remaining: 3m 19s
7:	learn: 0.6848549	total: 1.98s	remaining: 3m 15s
8:	learn: 0.6880505	total: 2.19s	remaining: 3m 12s
9:	learn: 0.6896552	total: 2.41s	remaining: 3m 10s
10:	learn: 0.6915222	total: 2.63s	remaining: 3m 8s
11:	learn: 0.6936908	total: 2.85s	remaining: 3m 6s
12:	learn: 0.6946474	total: 3.06s	remaining: 3m 5s
13:	learn: 0.6960450	total: 3.27s	remaining: 3m 3s
14:	learn: 0.6965604	total: 3.47s	remaining: 3m 1s
15:	learn: 0.6973869	total: 3.69s	remaining: 3m 1s
16:	learn: 0.6983792	total: 3.9s	remaining: 2m 59s
17:	learn: 0.6992691	total: 4.11s	remaining: 2m 58s
18:	learn: 0.6996635	total: 4.32s	remaining: 2m 57s
19:	learn: 0.7000766	total: 4.54s	

In [25]:
bestparams = {'depth': 5,
 'iterations': 250,
 'learning_rate': 0.1,
 'l2_leaf_reg': 6,
 'border_count': 128,
 'thread_count': 4,
 'task_type': 'GPU',
 'loss_function': 'Logloss',
 'eval_metric': 'AUC',
 'auto_class_weights': 'Balanced',
 'simple_ctr': None,
 'verbose': True}


clf_3 = catboost.CatBoostClassifier(**bestparams)
categorical_features_indices = [0,1,2,3,4,5,6,9,10,11]

labels = q_df['service_canceled']
dataset = q_df.drop(["service_canceled"], axis=1)
    
train_pool = catboost.Pool(dataset, labels, cat_features=categorical_features_indices)
clf_3.fit(train_pool)

0:	learn: 0.6735320	total: 352ms	remaining: 1m 27s
1:	learn: 0.6740924	total: 704ms	remaining: 1m 27s
2:	learn: 0.6760882	total: 1.05s	remaining: 1m 26s
3:	learn: 0.6785091	total: 1.4s	remaining: 1m 26s
4:	learn: 0.6787655	total: 1.75s	remaining: 1m 25s
5:	learn: 0.6792744	total: 2.11s	remaining: 1m 25s
6:	learn: 0.6806188	total: 2.46s	remaining: 1m 25s
7:	learn: 0.6867636	total: 2.8s	remaining: 1m 24s
8:	learn: 0.6900039	total: 3.14s	remaining: 1m 23s
9:	learn: 0.6924743	total: 3.49s	remaining: 1m 23s
10:	learn: 0.6943140	total: 3.84s	remaining: 1m 23s
11:	learn: 0.6963256	total: 4.17s	remaining: 1m 22s
12:	learn: 0.6977105	total: 4.5s	remaining: 1m 22s
13:	learn: 0.6992153	total: 4.87s	remaining: 1m 22s
14:	learn: 0.7005403	total: 5.21s	remaining: 1m 21s
15:	learn: 0.7015023	total: 5.57s	remaining: 1m 21s
16:	learn: 0.7025363	total: 5.88s	remaining: 1m 20s
17:	learn: 0.7033189	total: 6.22s	remaining: 1m 20s
18:	learn: 0.7040109	total: 6.58s	remaining: 1m 19s
19:	learn: 0.7045684	tota

In [26]:
bestparams = {'depth': 2,
 'iterations': 1000,
 'learning_rate': 0.05,
 'l2_leaf_reg': 2,
 'border_count': 128,
 'thread_count': 4,
 'task_type': 'GPU',
 'loss_function': 'Logloss',
 'eval_metric': 'AUC',
 'auto_class_weights': 'Balanced',
 'simple_ctr': None,
 'verbose': True}


clf_4 = catboost.CatBoostClassifier(**bestparams)
categorical_features_indices = [0,1,2,3,4,5,6,9,10,11]

labels = q_df['service_canceled']
dataset = q_df.drop(["service_canceled"], axis=1)
    
train_pool = catboost.Pool(dataset, labels, cat_features=categorical_features_indices)
clf_4.fit(train_pool)

0:	learn: 0.6411735	total: 148ms	remaining: 2m 27s
1:	learn: 0.6411818	total: 280ms	remaining: 2m 19s
2:	learn: 0.6443262	total: 417ms	remaining: 2m 18s
3:	learn: 0.6443262	total: 550ms	remaining: 2m 16s
4:	learn: 0.6443262	total: 686ms	remaining: 2m 16s
5:	learn: 0.6443262	total: 818ms	remaining: 2m 15s
6:	learn: 0.6443262	total: 954ms	remaining: 2m 15s
7:	learn: 0.6444366	total: 1.09s	remaining: 2m 15s
8:	learn: 0.6491638	total: 1.23s	remaining: 2m 14s
9:	learn: 0.6512060	total: 1.36s	remaining: 2m 14s
10:	learn: 0.6512085	total: 1.5s	remaining: 2m 14s
11:	learn: 0.6512093	total: 1.63s	remaining: 2m 14s
12:	learn: 0.6521442	total: 1.76s	remaining: 2m 13s
13:	learn: 0.6606153	total: 1.9s	remaining: 2m 13s
14:	learn: 0.6806042	total: 2.02s	remaining: 2m 12s
15:	learn: 0.6815947	total: 2.14s	remaining: 2m 11s
16:	learn: 0.6815022	total: 2.28s	remaining: 2m 11s
17:	learn: 0.6818951	total: 2.4s	remaining: 2m 10s
18:	learn: 0.6831521	total: 2.52s	remaining: 2m 10s
19:	learn: 0.6837345	tota

# TEST DATA EDA

In [27]:
q_df_test = pd.read_csv("/kaggle/input/ai4digigov2021/queue_dataset_test.csv")
q_df_test["date_converted"] = pd.to_datetime(q_df_test["date"], format='%Y-%m-%d')

In [28]:
# --- Impute NA values ---

# impute ages
q_df_test["customer_age_appl"].fillna(q_df_test["customer_age_appl"].mode().iloc[0], inplace=True)

# impute ticket hours
q_df_test["ticket_taking_time_hour"] = q_df_test['time_start_process'].str[:2] 
q_df_test["ticket_taking_time_hour"] = q_df_test["ticket_taking_time_hour"].astype(str)
q_df_test['ticket_taking_time_hour'].fillna(q_df_test['ticket_taking_time_hour'].mode().iloc[0], inplace= True)
q_df_test["ticket_taking_time_hour"] = q_df_test["ticket_taking_time_hour"].replace("nan", q_df_test['ticket_taking_time_hour'].mode().iloc[0])

In [29]:
# --- Extract Separate dates from date ---

# extract month 
q_df_test["approach_month"] = q_df_test["date_converted"].dt.month.astype(str)

# extract weekdays
week_days={0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
q_df_test['weekday'] = q_df_test['date_converted'].dt.dayofweek.map(week_days)

In [33]:
# --- Prepare Dataset For Prediction ---
q_df_test.drop(["date", "time_start_process", "date_converted"], axis=1, inplace=True) 
dataset = q_df_test 
dataset = dataset.loc[:, dataset.columns != 'id']

## Test Data Ensemble Prediction

In [34]:
y_pred_test_1 = clf_1.predict_proba(dataset)
y_pred_test_2 = clf_2.predict_proba(dataset)
y_pred_test_3 = clf_3.predict_proba(dataset)
y_pred_test_4 = clf_4.predict_proba(dataset)

In [35]:
y_scores_test = ((y_pred_test_1[:, 1] + y_pred_test_2[:, 1] + y_pred_test_3[:, 1])*3/4 + y_pred_test_4[:, 1]/4)/4

In [36]:
q_df_test['service_canceled'] = y_scores_test
q_df_test[["id", "service_canceled"]].to_csv("submission_4_catboost.csv", index=False)